<a href="https://colab.research.google.com/github/JasonD28/CS-175/blob/master/model/mask_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
|!pip install pydicom

In [2]:
%%shell
git clone https://github.com/JasonD28/CS-175.git
cd CS-175
cp model/dataset.py  ../

fatal: destination path 'CS-175' already exists and is not an empty directory.


In [0]:
import torch
from torch import optim
import torchvision
from dataset import PneumoniaDataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

root = "drive/My Drive/cs-175-data"

In [0]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                   hidden_layer,
                                                   num_classes=2)

In [7]:
%%shell

# Using PyTorch helper functions to simplify training
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.5.1

cp references/detection/utils.py ../
cp references/detection/engine.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 8311 (delta 3), reused 3 (delta 0), pack-reused 8297
Receiving objects: 100% (8311/8311), 10.23 MiB | 4.38 MiB/s, done.
Resolving deltas: 100% (5705/5705), done.
Branch 'v0.5.1' set up to track remote branch 'v0.5.1' from 'origin'.
Switched to a new branch 'v0.5.1'


In [29]:
%cd ../..

/content


In [0]:
from engine import train_one_epoch, evaluate
import utils

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
dataset = PneumoniaDataset(root, True)
test_dataset = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
test_dataset = torch.utils.data.Subset(test_dataset, indices[-50:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
lr_step = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)


In [5]:
for epoch in range(1):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_step.step()
    evaluate(model, test_data_loader, device=device)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch: [0]  [    0/13317]  eta: 5:41:51  lr: 0.000002  loss: 3.4368 (3.4368)  loss_classifier: 0.7682 (0.7682)  loss_box_reg: 0.0000 (0.0000)  loss_mask: 0.0000 (0.0000)  loss_objectness: 1.9983 (1.9983)  loss_rpn_box_reg: 0.6703 (0.6703)  time: 1.5403  data: 0.9781  max mem: 1645
Epoch: [0]  [   10/13317]  eta: 2:22:19  lr: 0.000012  loss: 3.0380 (2.8040)  loss_classifier: 0.5851 (0.5550)  loss_box_reg: 0.0000 (0.0045)  loss_mask: 0.0000 (0.0522)  loss_objectness: 1.7761 (1.5865)  loss_rpn_box_reg: 0.6625 (0.6058)  time: 0.6417  data: 0.0952  max mem: 2090
Epoch: [0]  [   20/13317]  eta: 2:12:57  lr: 0.000022  loss: 1.7833 (1.9253)  loss_classifier: 0.2438 (0.3464)  loss_box_reg: 0.0000 (0.0035)  loss_mask: 0.0000 (0.0380)  loss_objectness: 1.0095 (1.0309)  loss_rpn_box_reg: 0.5106 (0.5064)  time: 0.5529  data: 0.0058  max mem: 2090
Epoch: [0]  [   30/13317]  eta: 2:09:05  lr: 0.000032  loss: 0.4027 (1.3922)  loss_classifier: 0.0431 (0.2388)  loss_box_reg: 0.0000 (0.0033)  loss_mask: 

NameError: ignored